In [1]:
from transformers import pipeline, AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
import os
import pysolr
import requests
import json
import urllib
from deep_translator import GoogleTranslator
from langdetect import detect
from urllib.parse import urlencode
import string
import pandas as pd
import torch

In [14]:
line = "do you think climate change is real?"

In [15]:
punc = string.punctuation.replace("#", "")
query = line.translate(str.maketrans('', '', punc))
#query = line.strip('\n').replace(':', '').split(" ", maxsplit=1)
queryText = query.replace("\n","")
topic = "environment"

query = {
    "fl" : "* score",
    "q": f"body: {queryText} selftext: {queryText} topic: {topic}",
    "rows": 5,
    "defType": "edismax",
    "wt": "json",
    "qf": "body^1 selftext^1 topic^1"
}
result = urlencode(query)
inurl = 'http://34.134.102.49:8983/solr/IRF22P1/select?'+ result

data = urllib.request.urlopen(inurl).read()
docs = json.loads(data.decode('utf-8'))['response']['docs']

In [16]:
df = pd.DataFrame(docs)

In [17]:
df

,author,id,subreddit,full_link,is_submission,created_at,body,parent_body,parent_id,_version_,score
0,FixForb,hxzv9r0,Foodforthought,https://www.reddit.com/r/Foodforthought/commen...,False,2022-02-22T18:39:35Z,&gt;I can't help but wonder if we stopped talk...,"Perhaps an unpopular opinion, but I'm starting...",t1_hxzjf6v,1744516167577370626,17.561678
1,Usalien1,hylhyrx,conspiracy,https://www.reddit.com/r/conspiracy/comments/t...,False,2022-02-27T02:55:44Z,Chances are most of those are true. The global...,Has anyone else been watching CPAC (Conservati...,t3_t2e008,1744527106401894403,16.537197
2,half-a-duck,i05amfn,cscareerquestions,https://www.reddit.com/r/cscareerquestions/com...,False,2022-03-10T19:44:37Z,Most people prefer to blame heavy users of ele...,I tried Googling for this question a few times...,t1_i04q12r,1744251708265988098,15.038864
3,appleinwinter,i05x1bd,KUWTK,https://www.reddit.com/r/KUWTK/comments/tanb0k...,False,2022-03-10T22:12:44Z,"Let’s be real on climate change here, we’ve no...",To the stans: why?\n\nI've been wanting to wri...,t3_tanb0k,1744527105927938048,14.864832
4,DeadBirdLiveBird,hlcaeok,Foodforthought,https://www.reddit.com/r/Foodforthought/commen...,False,2021-11-20T02:48:10Z,Models are always wrong. Always. They're still...,&gt;As to the predictions: What about them? Th...,t1_hlc074p,1744516167834271745,14.694366


In [6]:
l = []
for i in range(len(df)):
  if isinstance(df['body'].iloc[i], str) and isinstance(df['parent_body'].iloc[i], str):
    l.append(df['parent_body'].iloc[i]+' '+df['body'].iloc[i])

In [7]:
l

["Perhaps an unpopular opinion, but I'm starting to believe this is why the conservative narrative is so appealing to so many Americans. The truth is, climate change *isn't* the cause of all our problems. Or rather, it isn't the *only* cause of many disasters. The problem with climate change is that it's too abstract - taking carbon out of the air might reduce the chances of a flood or a wildfire in a few decades, but it won't prevent the one forecast for next week.\n\nI can't help but wonder if we stopped talking about climate and instead focused on practical, real-world solutions, we might stumble our way into something resembling climate resilience, instead of getting caught up in abstract climate arguments without advancing the sort of revolutionary change required to address it. I'm not a climate denier, and I absolutely believe that we should be doing everything possible to reduce, counteract, and prevent climate change. But I also am starting to see that as a politically unworka

In [8]:
res = "<P> " + " <P> ".join(l)

In [9]:
res = res.replace("\n","")

In [10]:
res

"<P> Perhaps an unpopular opinion, but I'm starting to believe this is why the conservative narrative is so appealing to so many Americans. The truth is, climate change *isn't* the cause of all our problems. Or rather, it isn't the *only* cause of many disasters. The problem with climate change is that it's too abstract - taking carbon out of the air might reduce the chances of a flood or a wildfire in a few decades, but it won't prevent the one forecast for next week.I can't help but wonder if we stopped talking about climate and instead focused on practical, real-world solutions, we might stumble our way into something resembling climate resilience, instead of getting caught up in abstract climate arguments without advancing the sort of revolutionary change required to address it. I'm not a climate denier, and I absolutely believe that we should be doing everything possible to reduce, counteract, and prevent climate change. But I also am starting to see that as a politically unworkab

In [11]:
model_name = "yjernite/bart_eli5"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.to(device)

In [12]:
query_and_docs = "question: {} context: {}".format(queryText, res)

In [13]:
query_and_docs

"question: do you think climate change is real context: <P> Perhaps an unpopular opinion, but I'm starting to believe this is why the conservative narrative is so appealing to so many Americans. The truth is, climate change *isn't* the cause of all our problems. Or rather, it isn't the *only* cause of many disasters. The problem with climate change is that it's too abstract - taking carbon out of the air might reduce the chances of a flood or a wildfire in a few decades, but it won't prevent the one forecast for next week.I can't help but wonder if we stopped talking about climate and instead focused on practical, real-world solutions, we might stumble our way into something resembling climate resilience, instead of getting caught up in abstract climate arguments without advancing the sort of revolutionary change required to address it. I'm not a climate denier, and I absolutely believe that we should be doing everything possible to reduce, counteract, and prevent climate change. But I

In [14]:
model_input = tokenizer(query_and_docs, truncation=True, padding=True, return_tensors="pt")

generated_answers_encoded = model.generate(input_ids=model_input["input_ids"].to(device),
                                           attention_mask=model_input["attention_mask"].to(device),
                                           min_length=32,
                                           max_length=64,
                                           do_sample=False, 
                                           early_stopping=True,
                                           num_beams=2,
                                           temperature=1.0,
                                           top_k=None,
                                           top_p=None,
                                           eos_token_id=tokenizer.eos_token_id,
                                           no_repeat_ngram_size=3,
                                           num_return_sequences=1)
tokenizer.batch_decode(generated_answers_encoded, skip_special_tokens=True,clean_up_tokenization_spaces=True)

[" Yes, it's real. It's not a political issue, it is a scientific issue. The problem with climate change is that it's too expensive to do anything about it."]